In [19]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
#    if total_rule/total_records<0.4:
  #      return (999,);
    #0.02 0.5
    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(count/total_records)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    random.seed(42)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof



In [20]:
from sklearn.neighbors import KNeighborsClassifier

def bestIndividual_KNN(hof,df):
    k = 6
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/lym/lymphography.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [21]:
from sklearn.model_selection import StratifiedKFold


def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=500)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('data/lym/lymphography.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier


def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('data/lym/lymphography.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier(max_depth=6)
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [23]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,0.1,0.2,0.3,0.4,0.5]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=50
            n_gen=41
            # read dataframe from csv
            df = pd.read_csv('data/lym/lymphography.csv')
            # encode labels column to numbers
            y = df['target']
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")

            # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue

Accuracy with all features: 	(0.0,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg      	min
0  	50    	0.0604674	0  
1  	27    	0.0131334	0  
2  	31    	0.00403952	0  
3  	37    	0.0016375 	0  
4  	34    	0.000306995	0  
5  	31    	0.000698155	0  
6  	31    	0.000596269	0  
7  	34    	0.000428763	0  
8  	35    	0.00133904 	0  
9  	30    	0.000421847	0  
10 	21    	0          	0  
11 	22    	0.000143885	0  
12 	30    	0.000277831	0  
13 	37    	0.000413884	0  
14 	31    	0.000138889	0  
15 	29    	0.000307692	0  
16 	33    	0.000699522	0  
17 	27    	0.000434327	0  
18 	32    	0.00013986 	0  
19 	32    	0          	0  
20 	32    	0.000555635	0  
21 	35    	0.00198638 	0  
22 	31    	0.000138889	0  
23 	26    	0.00133333 	0  
24 	35    	0.000138889	0  
25 	23    	0.000988436	0  
26 	29    	0.000136986	0  
27 	31    	0.000286772	0  
28 	33    	0.000739872	0  
29 	25    	0.000274995	0  
30 	30    	0.000719424	0  
31 	31    	0.000138889	0  
32 	29    	0.00162602 	0  
33 	36    	0.00028169 	0  
34 	26    	0.000849122	0  
35 	34    	0.000568619	0  
36 	34   

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  UserWarning,


Accuracy with all features: 	(0.1,)



...

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  UserWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg     	min     
0  	50    	0.936129	0.532581
1  	27    	0.859551	0.532581
2  	31    	0.788482	0.518741
3  	37    	0.681483	0.455615
4  	34    	0.600777	0.455615
5  	31    	0.558881	0.455615
6  	31    	0.525364	0.455615
7  	34    	0.500259	0.455615
8  	35    	0.483727	0.455615
9  	30    	0.473952	0.455615
10 	21    	0.460104	0.455615
11 	22    	0.470783	0.455615
12 	30    	0.472938	0.455615
13 	37    	0.469298	0.387414
14 	31    	0.467622	0.387414
15 	29    	0.465544	0.387414
16 	33    	0.47399 	0.387414
17 	27    	0.471705	0.387414
18 	32    	0.438606	0.387414
19 	32    	0.408989	0.387414
20 	32    	0.418378	0.387414
21 	35    	0.407014	0.387414
22 	31    	0.39327 	0.387414
23 	26    	0.416753	0.387414
24 	35    	0.401375	0.387414
25 	23    	0.424086	0.387414
26 	29    	0.406625	0.387414
27 	31    	0.425896	0.387414
28 	33    	0.409288	0.387414
29 	25    	0.399673	0.387414
30 	30    	0.40989 	0.387414
31 	31    	0.394328	0.387414
32 	29    	0.419336	0.387414
33 	36    	0.4

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  UserWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	50    	1.01789	0.54204
1  	27    	0.923717	0.54204
2  	31    	0.833617	0.525497
3  	37    	0.702803	0.466426
4  	34    	0.615996	0.466426
5  	31    	0.569571	0.466426
6  	31    	0.535566	0.466426
7  	34    	0.510718	0.466426
8  	35    	0.495119	0.466426
9  	30    	0.48602 	0.466426
10 	21    	0.471266	0.466426
11 	22    	0.482337	0.466426
12 	30    	0.484884	0.466426
13 	37    	0.481608	0.400928
14 	31    	0.479487	0.400928
15 	29    	0.477423	0.400928
16 	33    	0.487004	0.400928
17 	27    	0.486745	0.400928
18 	32    	0.451484	0.400928
19 	32    	0.424812	0.400928
20 	32    	0.433459	0.400928
21 	35    	0.421865	0.400928
22 	31    	0.406959	0.400928
23 	26    	0.432875	0.400928
24 	35    	0.415173	0.400928
25 	23    	0.440492	0.400928
26 	29    	0.421166	0.400928
27 	31    	0.441733	0.400928
28 	33    	0.423828	0.400928
29 	25    	0.413849	0.400928
30 	30    	0.424606	0.400928
31 	31    	0.408125	0.400928
32 	29    	0.434444	0.400928
33 	36    	0.425652

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  UserWarning,


In [24]:
#seed 42 n_pop=50 n_gen=100 lamda=0
#['f3', 'f5', 'f9', 'f12', 'f14', 'f16'] KNN=0.8048 SVM=0.7852 DT=0.791

#seed 42 n_pop=50 n_gen=100 lamda=0.1
#['f3', 'f7', 'f12', 'f14', 'f18'] KNN=0.8114 SVM=0.8457 DT=0.8314

#seed 42 n_pop=50 n_gen=100 lamda=0.2
#['f3', 'f7', 'f12', 'f14', 'f18'] KNN=0.8114 SVM=0.8457 DT=0.8314

#seed 42 n_pop=50 n_gen=100 lamda=0.3
#['f3', 'f9', 'f12', 'f14', 'f16'] KNN=0.8048 SVM=0.7852 DT=0.791

#seed 42 n_pop=50 n_gen=100 lamda=0.4
#['f4', 'f11', 'f12', 'f14'] KNN=0.7705 SVM=0.7238 DT=0.7362

#seed 42 n_pop=50 n_gen=50 lamda=0.5
#['f4', 'f6', 'f8', 'f12'] KNN=0.5352 SVM=0.6419 DT=0.6348

#seed 42 n_pop=50 n_gen=50 lamda=0.6
#['f4', 'f6', 'f8', 'f12'] KNN=0.5352 SVM=0.6419 DT=0.6348

#seed 42 n_pop=50 n_gen=15 lamda=0.7
#['f4', 'f12'] KNN=0.5286 SVM=0.5086 DT=0.5081

#seed 42 n_pop=50 n_gen=15 lamda=0.8
#['f4', 'f12'] KNN=0.5286 SVM=0.5086 DT=0.5081

#seed 42 n_pop=50 n_gen=15 lamda=0.9
#['f4', 'f12'] KNN=0.5286 SVM=0.5086 DT=0.5081

#seed 42 n_pop=50 n_gen=15 lamda=1.0
#['f4', 'f12'] KNN=0.5286 SVM=0.5086 DT=0.5081
